In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [7]:
# ID da planilha
sheet_id = '1a2XaZct07CVfJwZSE6Ywl1VtblPurkgnVeebq94xaEg'
# IDs das abas
tab_ids = {'Offers':'0',
           'Bids':'2060141366'}

offers = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Offers"]}')

bids = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Bids"]}')

num_offers = len(offers)
num_bids = len(bids)

#Modelo Concreto
model = ConcreteModel()

#Set vendedores:
model.offers = Set(initialize=range(num_offers))

#Set compradores:
model.bids = Set(initialize=range(num_bids))

#Parâmetros dos vendedores:
model.selling_price = Param(model.offers, initialize={i: offers.loc[i, "$/MW"] for i in range(num_offers)})
model.selling_entity = Param(model.offers, initialize={i: offers.loc[i, "Entity"] for i in range(num_offers)},domain=Any)

#Parâmetros ds compradores:
model.buying_price = Param(model.bids, initialize={i: bids.loc[i, "$/MW"] for i in range(num_bids)})
model.buying_entity = Param(model.bids, initialize={i: bids.loc[i, "Entity"] for i in range(num_bids)})

#Variável de compra:
buying_max = bids['Load (MW)'].tolist()
buying_bounds = {i: (0, buying_max[i]) for i in range(num_bids)}
model.buy = Var(model.bids, bounds=buying_bounds, domain=Reals)

#Variável de venda:
selling_max = offers['P_max (MW)'].tolist()
selling_bounds = {i: (0, selling_max[i]) for i in range(num_offers)}
model.sell = Var(model.offers, bounds = selling_bounds, domain=Reals)

#Restrição:
model.restr_igualdade = ConstraintList()
model.restr_igualdade.add(expr=sum(model.buy[i] for i in model.bids) - sum(model.sell[i] for i in model.offers) == 0)

#FOB:
def fob(model):
    compra = sum(model.buy[i]*model.buying_price[i] for i in model.bids)
    venda = sum(model.sell[i]*model.selling_price[i] for i in model.offers)
    return compra-venda
model.fob = Objective(rule=fob, sense=maximize)
model.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('glpk')
results = solver.solve(model, tee=False)

In [8]:
#Exibir os resultados:
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', round(value(model.fob),2), '\n')
print('Resultados das Energias Vendidas:')
for i in model.offers:
    print(f'Lance {i+1} - Entidade {model.selling_entity[i]}: {model.sell[i].value} MWh')

print('Resultados das Energias Compradas:')
for i in model.bids:
    print(f'Lance {i+1} - Entidade {model.buying_entity[i]}: {model.buy[i].value} MWh')

# Obter o custo marginal (multiplicador de Lagrange) da restrição de igualdade
custo_marginal = model.dual[model.restr_igualdade[1]]
print("Custo Marginal: R$", custo_marginal)


Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 2430.0 

Resultados das Energias Vendidas:
Lance 1 - Entidade A: 20.0 MWh
Lance 2 - Entidade A: 40.0 MWh
Lance 3 - Entidade A: 0.0 MWh
Lance 4 - Entidade B: 0.0 MWh
Lance 5 - Entidade B: 10.0 MWh
Lance 6 - Entidade B: 0.0 MWh
Resultados das Energias Compradas:
Lance 1 - Entidade X: 0.0 MWh
Lance 2 - Entidade X: 20.0 MWh
Lance 3 - Entidade X: 5.0 MWh
Lance 4 - Entidade Y: 0.0 MWh
Lance 5 - Entidade Y: 30.0 MWh
Lance 6 - Entidade Y: 15.0 MWh
Custo Marginal: R$ 25.0
